#**COLLABORATIVE BASED RECOMMENDATION SYSTEM**

**A collaborative-based recommendation system predicts user preferences by leveraging the behavior and preferences of other users. It typically works by finding similarities between users (user-based) or items (item-based).**

**Collaborative Filtering:**

*A recommendation technique based on user-item interactions (e.g., ratings), assuming that similar users will like similar items.Python libraries like scikit-learn provide built-in functions for cosine similarity calculation.*

**Cosine Similarity:**

*A metric to calculate similarity between users (or items) by measuring the cosine of the angle between their rating vectors.*

**User-Item Matrix:**

*A matrix where rows represent users and columns represent items. Values are user ratings or interactions.*

**Cosine Similarity Calculation:**

*Cosine_similarity(A, B) returns a value between -1 and 1, where 1 indicates identical preferences and 0 indicates no similarity.*

**User-Based Collaborative Filtering:**

*Identifies users similar to the target user, then recommends items based on what similar users liked.*

**Item-Based Collaborative Filtering:**

*Finds similar items to those the target user has rated highly, and recommends those items.*

**Weighted Average:**

*For recommendations, weighted average of ratings is used from similar users/items.*





#**Cosine Similarity**

**Cosine similarity is a metric used to measure how similar two vectors (such as user ratings or item features) are, based on the cosine of the angle between them. It is often used in collaborative filtering recommendation systems to compare users or items. The formula for cosine similarity between two vectors
𝐴 and  𝐵 is given by:**

cosine similarity
(
𝐴
,
𝐵
)
=
𝐴
⋅
𝐵
∥
𝐴
∥
∥
𝐵
∥
cosine similarity(A,B)=
∥A∥∥B∥
A⋅B
​

**Where:**

𝐴
⋅
𝐵
is the dot product of the vectors
𝐴
and
𝐵
,
∥
𝐴
∥
and
∥
𝐵
∥
are the Euclidean norms (magnitude) of vectors
𝐴
and
𝐵
, respectively.
The dot product
𝐴
⋅
𝐵
is calculated as the sum of the element-wise multiplication of the two vectors, while the Euclidean norm of a vector
𝐴
is calculated as
𝐴
1
2
+
𝐴
2
2
+
⋯
+
𝐴
𝑛
2

**Cosine similarity ranges from -1 to 1:**

**1 indicates that the vectors are identical in orientation**

**0 indicates no similarity (the vectors are orthogonal or unrelated)**

**-1 indicates that the vectors are diametrically opposed**

# Importing Libraries and Dataset

In [262]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

#Reading DAT files into Dataframes

In [263]:
columns = ["movieid", "title", "genre"]
movies = pd.read_table("movies.dat", names = columns, sep = "::", encoding = "latin1", engine = "python")
movies

,movieid,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [264]:
columns = ['userid', 'movieid', 'rating', 'timestamp']
ratings = pd.read_table("ratings.dat", names = columns, sep = "::", encoding = "latin1", engine='python')
ratings

,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


#Shape of Dataframes

In [265]:
# movies

print('Number of rows:  ',movies.shape[0])
print('Number of columns:  ',movies.shape[1])

Number of rows:   3883
Number of columns:   3


In [266]:
# ratings

print('Number of rows:  ',ratings.shape[0])
print('Number of columns:  ',ratings.shape[1])

Number of rows:   1000209
Number of columns:   4


#Total count of ratings for each rating

In [267]:
ratings['rating'].value_counts().sort_index(ascending = False)

,count
rating,
5,226310
4,348971
3,261197
2,107557
1,56174


# Counts of ratings as per users

In [268]:
ratings.groupby('userid').size()

,0
userid,
1,53
2,129
3,51
4,21
5,198
...,...
6036,888
6037,202
6038,20


In [269]:
user_id_rating_count = ratings.groupby('userid').size()

In [270]:
user_id_rating_count.describe()

,0
count,6040.000000
mean,165.597517
std,192.747029
min,20.000000
25%,44.000000
50%,96.000000
75%,208.000000
max,2314.000000


#The 25th percentile of total number of ratings is 43 so, taking the users who have given more than 50 ratings

#Only keeping users who have given more than 50 ratings

In [271]:
user_id_rating_count[user_id_rating_count > 50].index

Index([   1,    2,    3,    5,    6,    8,    9,   10,   11,   13,
       ...
       6026, 6030, 6031, 6032, 6033, 6035, 6036, 6037, 6039, 6040],
      dtype='int64', name='userid', length=4247)

In [272]:
user_ids_with_more_than_50_ratings = user_id_rating_count[user_id_rating_count > 50].index

#Counts of movies as per viewers

In [273]:
ratings.groupby('movieid').size()

,0
movieid,
1,2077
2,701
3,478
4,170
5,296
...,...
3948,862
3949,304
3950,54


In [274]:
movie_id_rating_count = ratings.groupby('movieid').size()

#Only keeping movies which has received more than 50 ratings

In [275]:
movie_id_rating_count[movie_id_rating_count > 50].index

Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
       ...
       3932, 3936, 3937, 3943, 3946, 3947, 3948, 3949, 3950, 3952],
      dtype='int64', name='movieid', length=2499)

In [276]:
movie_ids_with_more_than_50_ratings = movie_id_rating_count[movie_id_rating_count > 50].index

#Taking both users who have given more than 50 ratings and movies which received more than 50 ratings

In [277]:
ratings = ratings[(ratings.userid.isin(user_ids_with_more_than_50_ratings)) & (ratings.movieid.isin(movie_ids_with_more_than_50_ratings))]

# Ratings modified Dataframe

In [278]:
ratings.head()

,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# The shape of Ratings modified Dataframe

In [279]:
print('Number of rows:  ',ratings.shape[0])
print('Number of columns:  ',ratings.shape[1])

Number of rows:   918946
Number of columns:   4


#Resetting the index for the Dataframe

In [280]:
ratings=ratings.reset_index(drop=True)
print(ratings.shape)
ratings.head()

(918946, 4)


,userid,movieid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#Merging the two dataframes
#movies & modified ratings dataframes

In [281]:
merged = pd.merge(movies, ratings, on='movieid')
merged

,movieid,title,genre,userid,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474
...,...,...,...,...,...,...
918941,3952,"Contender, The (2000)",Drama|Thriller,5812,4,992072099
918942,3952,"Contender, The (2000)",Drama|Thriller,5831,3,986223125
918943,3952,"Contender, The (2000)",Drama|Thriller,5837,4,1011902656
918944,3952,"Contender, The (2000)",Drama|Thriller,5927,1,979852537


#'timestamp' column is not required
# Dropping timestamp column

In [282]:
merged = pd.merge(movies, ratings, on='movieid').drop(['timestamp'], axis = 1)
merged

,movieid,title,genre,userid,rating
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5
...,...,...,...,...,...
918941,3952,"Contender, The (2000)",Drama|Thriller,5812,4
918942,3952,"Contender, The (2000)",Drama|Thriller,5831,3
918943,3952,"Contender, The (2000)",Drama|Thriller,5837,4
918944,3952,"Contender, The (2000)",Drama|Thriller,5927,1


#Creating a pivot table for users and movies

In [283]:
user_ratings = merged.pivot_table(index = ['userid'], columns = ['title'], values = 'rating')
user_ratings

title,'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",187 (1997),...,You've Got Mail (1998),Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
userid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
6036,3.0,NaN,NaN,NaN,2.0,4.0,NaN,NaN,1.0,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,3.0,NaN,NaN,2.0
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Filling the null values with zeros

In [284]:
user_ratings = user_ratings.fillna(0, axis=1)
user_ratings

title,'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",187 (1997),...,You've Got Mail (1998),Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
userid,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6036,3.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Creating a transpose to make users as columns and movies as rows

In [285]:
user_ratings_T = user_ratings.T
user_ratings_T

userid,1,2,3,5,6,8,9,10,11,13,...,6026,6030,6031,6032,6033,6035,6036,6037,6039,6040
title,,,,,,,,,,,,,,,,,,,,,
'Night Mother (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Young Poisoner's Handbook, The (1995)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Young Sherlock Holmes (1985),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0
Your Friends and Neighbors (1998),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#Converting the 'user_ratings_T' to a sparse matrix
#Sparse matrix improve efficiency in computations involving large matrices with many zero entries.

In [286]:
sparse.csr_matrix(user_ratings_T.values)

<2499x4247 sparse matrix of type '<class 'numpy.float64'>'
	with 918946 stored elements in Compressed Sparse Row format>

In [287]:
sparse_df = sparse.csr_matrix(user_ratings_T.values)

#Checking the cosine similarity within the sparse matrix

In [288]:
cosine_similarity(sparse_df)

array([[1.        , 0.12216203, 0.12098183, ..., 0.13952341, 0.076536  ,
        0.08180941],
       [0.12216203, 1.        , 0.07923412, ..., 0.08327461, 0.08312361,
        0.06818722],
       [0.12098183, 0.07923412, 1.        , ..., 0.14346861, 0.1670255 ,
        0.11767759],
       ...,
       [0.13952341, 0.08327461, 0.14346861, ..., 1.        , 0.21011001,
        0.15626356],
       [0.076536  , 0.08312361, 0.1670255 , ..., 0.21011001, 1.        ,
        0.25188157],
       [0.08180941, 0.06818722, 0.11767759, ..., 0.15626356, 0.25188157,
        1.        ]])

#Creating a matrix with cosine similarity within the sparse matrix

In [289]:
corrMatrix = pd.DataFrame(cosine_similarity(sparse_df), index=user_ratings.columns, columns=user_ratings.columns)
corrMatrix

title,'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",187 (1997),...,You've Got Mail (1998),Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Your Friends and Neighbors (1998),Zero Effect (1998),eXistenZ (1999)
title,,,,,,,,,,,,,,,,,,,,,
'Night Mother (1986),1.000000,0.122162,0.120982,0.170056,0.082781,0.146202,0.102241,0.118808,0.049644,0.066272,...,0.089716,0.222468,0.135407,0.079762,0.039005,0.054354,0.090263,0.139523,0.076536,0.081809
'Til There Was You (1997),0.122162,1.000000,0.079234,0.047960,0.135057,0.109357,0.136414,0.082571,0.053200,0.000000,...,0.165604,0.077064,0.052534,0.062434,0.052120,0.030682,0.066238,0.083275,0.083124,0.068187
"'burbs, The (1989)",0.120982,0.079234,1.000000,0.138382,0.199097,0.257651,0.188871,0.174067,0.205921,0.103708,...,0.230887,0.148730,0.241060,0.305238,0.238053,0.117727,0.216358,0.143469,0.167025,0.117678
...And Justice for All (1979),0.170056,0.047960,0.138382,1.000000,0.081477,0.200244,0.119955,0.210849,0.125288,0.129530,...,0.114044,0.139129,0.198796,0.128440,0.095046,0.095137,0.162117,0.115349,0.116972,0.120366
10 Things I Hate About You (1999),0.082781,0.135057,0.199097,0.081477,1.000000,0.204766,0.178196,0.163580,0.268506,0.071435,...,0.318791,0.084864,0.191808,0.202448,0.183936,0.071405,0.151223,0.083399,0.187519,0.175641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Young Poisoner's Handbook, The (1995)",0.054354,0.030682,0.117727,0.095137,0.071405,0.106295,0.025379,0.089702,0.060422,0.030302,...,0.063378,0.060369,0.143616,0.113786,0.082532,1.000000,0.087714,0.130206,0.181974,0.120670
Young Sherlock Holmes (1985),0.090263,0.066238,0.216358,0.162117,0.151223,0.230313,0.126140,0.171494,0.219508,0.057332,...,0.193437,0.138068,0.321171,0.346073,0.204226,0.087714,1.000000,0.094550,0.172592,0.188293
Your Friends and Neighbors (1998),0.139523,0.083275,0.143469,0.115349,0.083399,0.129421,0.062223,0.123149,0.060021,0.043144,...,0.111702,0.142524,0.150783,0.115211,0.083605,0.130206,0.094550,1.000000,0.210110,0.156264


#Getting the most viewed movies related to the given movie

#For a single movie

In [290]:
def get_similar_movies(movie_name):

    similar_score = corrMatrix[movie_name].sort_values(ascending=False)

    return similar_score

In [291]:
get_similar_movies('Superman II (1980)').head(15)

,Superman II (1980)
title,
Superman II (1980),1.000000
Superman (1978),0.684607
Superman III (1983),0.656253
Star Trek III: The Search for Spock (1984),0.610687
Star Trek: The Wrath of Khan (1982),0.591920
Star Trek IV: The Voyage Home (1986),0.591851
Robocop (1987),0.570356
Tron (1982),0.562766
"Running Man, The (1987)",0.558785


#For a group of movies

In [292]:
romantic_lover = ["'Night Mother (1986)","Young Poisoner's Handbook, The (1995)","Aliens (1986)","2001: A Space Odyssey (1968)"]
similar_movies = pd.DataFrame()

for movie in romantic_lover:

    # Get recommendations for each movie
    temp_df = get_similar_movies(movie)

    # Concatenate with existing similar_movies (fixing the deprecation warning)
    similar_movies = pd.concat([similar_movies, temp_df], axis = 1 )

similar_movies = similar_movies.T

similar_movies.mean().sort_values(ascending=False).head(20)

,0
2001: A Space Odyssey (1968),0.467192
Aliens (1986),0.458240
Alien (1979),0.410131
Blade Runner (1982),0.394549
Star Wars: Episode V - The Empire Strikes Back (1980),0.394319
"Terminator, The (1984)",0.392871
Star Wars: Episode IV - A New Hope (1977),0.387722
Close Encounters of the Third Kind (1977),0.383676
"Clockwork Orange, A (1971)",0.371008
Back to the Future (1985),0.367048


By leveraging user ratings or behavior data, cosine similarity measures the degree of similarity between users or items, allowing the system to identify and recommend products or services that are most relevant to the individual.

Scikit-learn, make it easier to handle large datasets and perform necessary calculations efficiently. While the user-based collaborative filtering approach works well in many cases, the system can also be adapted to item-based recommendations depending on the data's nature.